In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 20 10:49:52 2020

@author: cleme
"""
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing


# Import data
#DATA_TRAIN_PATH = "../train_data/features_no_segmentation.csv"
#DATA_TRAIN_PATH = r"C:\Users\cleme\Documents\EPFL\SV MA1\ML\ML_Project2\train_data\features_no_segmentation.csv"
FEATURES_PATH = Path("../data/features_no_segmentation.csv")
LABELS_PATH = Path("../data/labels_no_segmentation.csv")
coughs = pd.read_csv(FEATURES_PATH, sep=',', index_col = ['Unnamed: 0','File_Name'])
labels = pd.read_csv(LABELS_PATH, sep=',', index_col = ['Unnamed: 0','File_Name'])

In [ ]:
# Have an insight
coughs.head()
#coughs.loc[:,'EEPD50_100':'Expert'].shape[1]
labels.head()

In [ ]:
# Check the number of undefined values -> should be 0
coughs.isnull().sum().sum()

In [ ]:
### Distribution exploration before standardization (not necessary to run)

%matplotlib inline

import matplotlib.pyplot as plt


nrows = 14
ncols = 5
fig, axes = plt.subplots(nrows, ncols)

feature_id = 0
for row in range(nrows):
    for col in range(ncols):
        if feature_id == 67: break
        feature_tbp  = coughs.iloc[:,feature_id]
        axes[row, col].hist(feature_tbp[~np.isnan(feature_tbp)], bins='auto')
        axes[row, col].set_title('Feature: {}'.format(feature_id))
        feature_id += 1
    
    
fig.set_size_inches(19.5, 25)


In [ ]:
### Standardization

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(coughs.loc[:,'EEPD50_100':'PSD_3100-3700'])
new = pd.DataFrame(scaler.transform(coughs.loc[:,'EEPD50_100':'PSD_3100-3700']), index=(coughs.loc[:,'EEPD50_100':'PSD_3100-3700']).index, columns=(coughs.loc[:,'EEPD50_100':'PSD_3100-3700']).columns)

In [ ]:
coughs_std = pd.merge(new, coughs[['Expert', 'Age', 'Gender', 'Resp_Condition', 'Symptoms']], left_index=True, right_index=True)

In [ ]:
coughs_std.shape

In [ ]:
### Distribution exploration after standardization (not necessary to run)

nrows = 15
ncols = 5
fig, axes = plt.subplots(nrows, ncols)

feature_id = 0
for row in range(nrows):
    for col in range(ncols):
        if feature_id == 67: break
        feature_tbp  = coughs_std.iloc[:,feature_id]
        axes[row, col].hist(feature_tbp[~np.isnan(feature_tbp)], bins='auto')
        axes[row, col].set_title('Feature: {}'.format(feature_id))
        feature_id += 1
    
    
fig.set_size_inches(19.5, 25)


In [ ]:
"""enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc.fit(coughs.loc[:,'Expert':])
enc.transform(coughs.loc[:,'Expert':])"""
# Categorical features handling... Not complete

In [ ]:
### Split data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(coughs_std, labels, test_size=0.2)

In [ ]:
### Logistic regression

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=0, max_iter=500).fit(X_train, y_train.to_numpy().squeeze())
predLR = LR.predict(X_test)

accuracyLR = LR.score(X_test, y_test.to_numpy().squeeze())

aucLR = roc_auc_score(y_test, predLR)
print(accuracyLR, aucLR)


In [ ]:
### Support Vector Machine Classifier

from sklearn.svm import SVC

SVM = SVC().fit(X_train, y_train.to_numpy().squeeze())
predSVM = SVM.predict(X_test)

accuracySVM = SVM.score(X_test, y_test.to_numpy().squeeze())

aucSVM = roc_auc_score(y_test, predSVM)
print(accuracySVM, aucSVM)

In [ ]:
### Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis().fit(X_train, y_train.to_numpy().squeeze())
predLDA = LDA.predict(X_test)

accuracyLDA = LDA.score(X_test, y_test.to_numpy().squeeze())
aucLDA = roc_auc_score(y_test, predLDA)
print(accuracyLDA, aucLDA)

In [ ]:
### K-Neighbors Classifier

from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=15, weights='uniform').fit(X_train, y_train.to_numpy().squeeze())
predKNN = KNN.predict(X_test)
accuracyKNN = KNN.score(X_test, y_test.to_numpy().squeeze())
aucKNN = roc_auc_score(y_test, predKNN)
print(accuracyKNN, aucKNN)

In [ ]:
### Gaussian Naive Bayes

from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB().fit(X_train, y_train.to_numpy().squeeze())
predGNB = GNB.predict(X_test)
accuracyGNB = GNB.score(X_test, y_test.to_numpy().squeeze())
aucGNB = roc_auc_score(y_test, predGNB)
print(accuracyGNB, aucGNB)

In [ ]:
### Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier

DTC = GaussianNB().fit(X_train, y_train.to_numpy().squeeze())
predDTC = DTC.predict(X_test)
accuracyDTC = DTC.score(X_test, y_test.to_numpy().squeeze())
aucDTC = roc_auc_score(y_test, predDTC)
print(accuracyDTC, aucDTC)

In [ ]:
### Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier() # parameters signification ?
RFC.fit(X_train, y_train.to_numpy().squeeze())
predRFC = RFC.predict(X_test)
accuracyRFC = RFC.score(X_test, y_test.to_numpy().squeeze())
aucRFC = roc_auc_score(y_test, predRFC) 

print(accuracyRFC, aucRFC)

In [ ]:
### XGBoost --> documentation on the web... need to investigate.